In [ ]:
### imports

# external modules
import os
import sys
import json
import numpy as np
from fnmatch import fnmatch
import importlib

# local modules
thisdir = os.getcwd()
topdir = os.path.abspath(os.path.join(thisdir, '../'))
sys.path.append(topdir)

import tools.iotools as iotools
import tools.dftools as dftools
import tools.omstools as omstools
from tools.omsapi.get_oms_data import get_oms_api
from tools.omsapi.get_oms_data import get_oms_data
from tools.omsapi.get_oms_data import get_oms_response_attribute
from tools.omsapi.get_oms_data import get_oms_response_attributes
from tools.omsapi.examples import get_lumisection_info

In [ ]:
# get the omsapi instance

omsapi = get_oms_api()

In [ ]:
# find runs that are present in the DQMIO data (for filtering the OMS response)

# define settings for which files to retrieve the run numbers from
datadir = '/eos/user/l/llambrec/dialstools-output'
dataset = 'ZeroBias'
reco = 'PromptReco'
me = 'PixelPhase1-Phase1_MechanicalView-PXBarrel-clusters_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_1'

# find files corresponding to settings
filepattern = f'{dataset}-Run*-{reco}-*-DQMIO-{me}.parquet'
files = sorted([os.path.join(datadir, f) for f in os.listdir(datadir) if fnmatch(f, filepattern)])

# retrieve run numbers
dqmio_runs = []
print(f'Reading {len(files)} files for retrieving run numbers...')
for f in files:
    df = iotools.read_parquet(f, columns=['run_number'])
    run_numbers = dftools.get_runs(df, runcolumn='run_number')
    dqmio_runs += run_numbers
dqmio_runs = sorted(list(set(dqmio_runs)))
dqmio_runs = np.array(dqmio_runs).astype(int)
print(f'Found {len(dqmio_runs)} runs.')

In [ ]:
# find out which attributes are available per lumisection

runnb = 378142 # dummy run, should not matter
ls_info = get_oms_data( omsapi, 'lumisections', runnb )
available_attributes = sorted(list(get_oms_response_attributes(ls_info)))
print(available_attributes)

In [ ]:
# define attributes to retrieve

attributes = [
    'start_time',
    'end_time',
    'run_number',
    'lumisection_number',
]

# quick check
for attribute in attributes:
    if attribute not in available_attributes:
        print(f'WARNING: attribute {attribute} does not seem to be available.')

In [ ]:
# set run range (maybe later split per era?)

runrange = (378142, 387721)

In [ ]:
# get the data from OMS

ls_info = get_lumisection_info(omsapi, runrange, attributes, run_filter=dqmio_runs)

In [ ]:
# save to json files

outputdir = 'omsdata'
if not os.path.exists(outputdir): os.makedirs(outputdir)
outputfile = os.path.join(outputdir, 'omsdata_lumisection_timestamps.json')
with open(outputfile, 'w') as f:
    json.dump(ls_info, f)
print(f'Created file {outputfile}')